### 写在前面

In [ ]:
'''
在使用随机森林等机器学习算法预测土壤属性的过程中，首先需要将所有栅格变量对齐（坐标系、分辨率、范围、行数、列数等），
然后，需要为多个对齐的栅格创建堆栈（堆叠或者stack），这样才能进行最后的预测成图。
但是，使用ArcMap等软件进行操作比较麻烦，于是产生了写一个Python脚本的想法。

时间：2024年04月28日 19:58:49
地点：海南海口
'''

### 思路

In [ ]:
'''
1.首先，检查不同栅格的坐标系，如有不同，将坐标系统一；
2.其次，将所有栅格重采样为目标分辨率；
3.然后，统一Extent（四个点）
3.然后，获取所有栅格相交的部分，制作掩膜，进行裁剪；
4.最后，处理异常值
'''

In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *
import os

### 对齐之前探索数据

In [2]:
# 设置环境
env.workspace =  r"F:\arcpy_resample\ToResample" 
env.overwriteOutput = True

# 读取文件夹内的所有tif文件
raster_files = arcpy.ListRasters("*", "TIF")

for raster in raster_files:
    columns = arcpy.management.GetRasterProperties(raster,"COLUMNCOUNT" )
    rows = arcpy.management.GetRasterProperties(raster,"ROWCOUNT" )
    tops = arcpy.management.GetRasterProperties(raster,"TOP" )
    bottoms = arcpy.management.GetRasterProperties(raster,"BOTTOM" )
    lefts = arcpy.management.GetRasterProperties(raster,"LEFT" )
    rights = arcpy.management.GetRasterProperties(raster,"RIGHT" )
    res_xs = arcpy.management.GetRasterProperties(raster,"CELLSIZEX" )
    res_ys = arcpy.management.GetRasterProperties(raster,"CELLSIZEY" )
    #print(f"栅格 {raster}")
    print(f"行数为{ rows}，列数为 {columns}, 北为{tops}, 南为{bottoms}, 东为{lefts}, 西为{rights}, 分辨率为{res_xs}*{res_ys}")

行数为3457，列数为 2823, 北为2211854.19925067, 南为2142750.25877813, 东为365817.979186296, 西为422248.532832227, 分辨率为19.9895691271452*19.9895691271452
行数为7017，列数为 6079, 北为2212203.107, 南为2142423.336404, 东为365296.243, 西为422554.234578, 分辨率为9.418982*9.94438799999999
行数为3456，列数为 2823, 北为2211854.19925067, 南为2142750.13033741, 东为365817.979186296, 西为422264.966032977, 分辨率为19.9953903105492*19.9953903105492
行数为3457，列数为 2823, 北为2211854.19925067, 南为2142750.25877813, 东为365817.979186296, 西为422248.532832227, 分辨率为19.9895691271452*19.9895691271452
行数为3460，列数为 2822, 北为2211950, 南为2142750, 东为365813.15625, 西为422253.15625, 分辨率为20*20
行数为3460，列数为 2822, 北为2211950, 南为2142750, 东为365813.15625, 西为422253.15625, 分辨率为20*20
行数为3457，列数为 2823, 北为2211854.19925067, 南为2142750.25877813, 东为365817.979186296, 西为422248.532832227, 分辨率为19.9895691271452*19.9895691271452
行数为3457，列数为 2823, 北为2211862.517, 南为2142722.517, 东为365817.68, 西为422277.68, 分辨率为20*20
行数为7017，列数为 6079, 北为2212198.903, 南为2142419.132404, 东为365296.633, 西为422554.636736, 分辨率为9.418984*9

### 检查坐标系是否统一

In [3]:
# 检查文件夹下所有栅格的坐标系是否统一
rasters_crs = []
for raster_file in raster_files:
    raster_crs = arcpy.Describe(raster_file).spatialReference.name
    rasters_crs.append(raster_crs)
def are_all_crs_same(lst):
    return all(crs == lst[0] for crs in lst)
print(are_all_crs_same(rasters_crs))
# print(rasters_crs)

True


### 重采样

In [4]:
# 将所有栅格重采样到目标分辨率
import time
start = time.time()

env.workspace = r"F:\arcpy_resample\ToResample" 
env.overwriteOutput = True

resampled_files =  r"F:\arcpy_resample\resampled" 

for raster_file in raster_files:
    # 构建输出文件的路径
    output_file = os.path.join(resampled_files, os.path.splitext(raster_file)[0] + "_rsp.tif")
    # 执行重采样
    arcpy.management.Resample(os.path.join(arcpy.env.workspace, raster_file),
                                                    output_file, "20 20", "BILINEAR")
print("Done!")
print((time.time() - start), "秒")

Done!
19.743176460266113 秒


In [9]:
# 读取重采样后的文件的行数和列数，找出模版栅格(行数列数最小)

env.workspace = r"F:\arcpy_resample\resampled" 
env.overwriteOutput = True
resampled_files = arcpy.ListRasters("*", "TIF")

for raster in resampled_files:
    columns = arcpy.management.GetRasterProperties(raster,"COLUMNCOUNT" )
    rows = arcpy.management.GetRasterProperties(raster,"ROWCOUNT" )
    tops = arcpy.management.GetRasterProperties(raster,"TOP" )
    bottoms = arcpy.management.GetRasterProperties(raster,"BOTTOM" )
    lefts = arcpy.management.GetRasterProperties(raster,"LEFT" )
    rights = arcpy.management.GetRasterProperties(raster,"RIGHT" )
    res_xs = arcpy.management.GetRasterProperties(raster,"CELLSIZEX" )
    res_ys = arcpy.management.GetRasterProperties(raster,"CELLSIZEY" )
    
    print(f"栅格为{ raster}, 行数为{ rows}，列数为 {columns}, 北为{tops}, 南为{bottoms}, 东为{lefts}, 西为{rights}, 分辨率为{res_xs}*{res_ys}", end = "\n\n")

栅格为Clay_Index_rsp.tif, 行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20

栅格为CR1022_NEW_rsp.tif, 行数为3489，列数为 2863, 北为2212203.107, 南为2142423.107, 东为365296.243, 西为422556.243, 分辨率为20*20

栅格为FlowPathLength_rsp.tif, 行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20

栅格为GRVI_rsp.tif, 行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20

栅格为LS_Factor_rsp.tif, 行数为3460，列数为 2822, 北为2211950, 南为2142750, 东为365813.15625, 西为422253.15625, 分辨率为20*20

栅格为MRRTF_rsp.tif, 行数为3460，列数为 2822, 北为2211950, 南为2142750, 东为365813.15625, 西为422253.15625, 分辨率为20*20

栅格为NDMI_rsp.tif, 行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20

栅格为RVI1209_rsp.tif, 行数为3489，列数为 2863, 北为2212198.903, 南为2142418.903, 东为365296.633, 西为422556.633, 分辨率为20*20

栅格为RVI1221_rsp.tif, 行数为3489，列数为 2863, 北为2212200.345, 南为21

### 对齐Extent

In [10]:
# 将重采样栅格的Extent对齐
# 参考 https://mp.weixin.qq.com/s/LJAQFqf7aUmMzbuDYCJHxQ
import time
start = time.time()

# 设置裁剪模版文件
template_file = r"F:\arcpy_resample\resampled\GRVI_rsp.tif"#行列数最小的栅格为模版

arcpy.env.workspace =  r"F:\arcpy_resample\resampled" 
env.overwriteOutput = True

resampled_files = arcpy.ListRasters("*", "TIF")

arcpy.env.snapRaster = template_file

aligned_file = r"F:\arcpy_resample\Aligned/" 
for raster in resampled_files:
    key_name = raster.split(".tif")[0] + "_aligned.tif"
    arcpy.management.Clip(raster, "#", aligned_file + key_name, template_file , "#", "#", "MAINTAIN_EXTENT")
    
print("Done!")
print((time.time() - start), "秒")

Done!
17.798445224761963 秒


In [11]:
# 重采样和对齐后，检查是否成功
arcpy.env.workspace =  r"F:\arcpy_resample\Aligned" 
aligned_files = arcpy.ListRasters("*", "TIF")

for raster in aligned_files:
    # Rows & Columns
    column = arcpy.management.GetRasterProperties(raster,"COLUMNCOUNT" )
    row = arcpy.management.GetRasterProperties(raster,"ROWCOUNT" )
    # Extent
    top = arcpy.management.GetRasterProperties(raster,"TOP" )
    bottom = arcpy.management.GetRasterProperties(raster,"BOTTOM" )
    left = arcpy.management.GetRasterProperties(raster,"LEFT" )
    right = arcpy.management.GetRasterProperties(raster,"RIGHT" )
    # Res
    res_x = arcpy.management.GetRasterProperties(raster,"CELLSIZEX" )
    res_y = arcpy.management.GetRasterProperties(raster,"CELLSIZEY" )
    print(f"行数为{ row}，列数为 {column}, 北为{top}, 南为{bottom}, 东为{left}, 西为{right}, 分辨率为{res_x}*{res_y}")

行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.

虽然检查得到的行数和列数，范围都是相同的，但在ArcMap里面查看的时候却发现边缘还是有对不齐的情况，不知道为什么这样

### 掩膜提取

In [12]:
# 提取所有栅格的交集，作为掩膜
env.workspace = r"F:\arcpy_resample\Aligned" 
env.overwriteOutput = True
# 列出所有的栅格文件

rasters = arcpy.ListRasters("*", "TIF")
# 构建 IsNull 表达式
if rasters:
    # 开始构建表达式，检查第一个栅格是否为空
    expression = IsNull(Raster(rasters[0]))
    
    # 为列表中的其他栅格添加更多的 IsNull 检查
    for raster in rasters[1:]:
        expression |= IsNull(Raster(raster))

    # 使用 SetNull 根据 expression 设置空值
    result_raster = SetNull(expression, 1)

    # 保存结果栅格
    result_raster.save(r"F:\arcpy_resample\Mask\Clip_mask.tif")
else:
    print("No rasters found in the workspace.")

### 掩膜裁剪，对齐像元分布

In [13]:
#最终将像元对齐
import time
start = time.time()

# 设置工作环境和覆盖输出设置
arcpy.env.workspace =  r"F:\arcpy_resample\Aligned"
env.overwriteOutput = True

# 设置裁剪模版文件和输出目录
template_file = r"F:\arcpy_resample\Mask\Clip_mask.tif"#行列数最小的栅格为模版
masked_file = r"F:\arcpy_resample\Aligned\mask" 

# 获取所有栅格文件
rasters = arcpy.ListRasters("*", "TIF")

for raster in rasters:
    key_name = raster.split(".tif")[0] + "_masked.tif"
    outExtractByMask = ExtractByMask(raster, template_file )
    out_path = masked_file + "\\" + key_name  # 构建完整输出路径
    outExtractByMask.save(out_path)
print("Done!")
print((time.time() - start), "秒")

Done!
12.597680568695068 秒


### 最终检查

In [14]:
# 掩膜裁剪后，检查是否成功
arcpy.env.workspace =   r"F:\arcpy_resample\Aligned\mask" 
masked_files = arcpy.ListRasters("*", "TIF")

for raster in masked_files:
    # Rows & Columns
    column = arcpy.management.GetRasterProperties(raster,"COLUMNCOUNT" )
    row = arcpy.management.GetRasterProperties(raster,"ROWCOUNT" )
    # Extent
    top = arcpy.management.GetRasterProperties(raster,"TOP" )
    bottom = arcpy.management.GetRasterProperties(raster,"BOTTOM" )
    left = arcpy.management.GetRasterProperties(raster,"LEFT" )
    right = arcpy.management.GetRasterProperties(raster,"RIGHT" )
    # Res
    res_x = arcpy.management.GetRasterProperties(raster,"CELLSIZEX" )
    res_y = arcpy.management.GetRasterProperties(raster,"CELLSIZEY" )
    print(f"行数为{ row}，列数为 {column}, 北为{top}, 南为{bottom}, 东为{left}, 西为{right}, 分辨率为{res_x}*{res_y}")

行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.19925067, 南为2142754.19925067, 东为365817.979186296, 西为422257.979186296, 分辨率为20*20
行数为3455，列数为 2822, 北为2211854.

经过检查（行数、列数、Extent、分辨率以及最重要的像元分布都对应上了，可以进行堆栈）

总结：
1.Extent是指栅格的最南最北、最东最西四个点的位置，不能表示每个像元的分布；
2.栅格的对齐其实是指每个像元的对齐，只要每个像元大小固定，像元分布也对齐，那么行数、列数、Extent等也就自然而然对齐了；
3.在研究之前，最好将研究区裁剪为矩形，等到最后一步再将得到的结果用研究区域行政区图裁剪，规则的矩形可以避免很多麻烦